In [1]:
import sys
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures



data = pd.read_csv("GenreData.csv")
df = pd.DataFrame(data)

X = data.drop(['title','artist','year','top genre'], axis = 1)
y = data['top genre']

#one-hot encoding
y_dum = pd.get_dummies(y)
scaler = MinMaxScaler()
normalized_class = scaler.fit_transform(y_dum) 				# returns an array
y_onehot = pd.DataFrame(normalized_class, columns = y_dum.columns)	# convert array to DataFrame
print("Pre-processed class :")
print(y_onehot)

#Normalize each fields of input
scaler = MinMaxScaler(feature_range=(0, 1))
X_rescaled = scaler.fit_transform(X)
X_rescaled = pd.DataFrame(data = X_rescaled, columns = X.columns)
print(X_rescaled)

# Combine the normalized input features and the target variable
data1 = pd.concat([X_rescaled, y], axis=1)			#data without onehot-encoding class
data = pd.concat([X_rescaled, y_onehot], axis=1)	#data with onehot-encoding class

"""
# Create a pair plot
sns.set(style="whitegrid")
g = sns.pairplot(data=data1, hue="top genre", corner=True, palette='husl')
plt.show()
"""

Pre-processed class :
     acoustic blues  adult standards  afrobeat  afropop  album rock  \
0               0.0              1.0       0.0      0.0         0.0   
1               0.0              0.0       0.0      0.0         0.0   
2               0.0              1.0       0.0      0.0         0.0   
3               0.0              1.0       0.0      0.0         0.0   
4               0.0              0.0       0.0      0.0         0.0   
..              ...              ...       ...      ...         ...   
561             0.0              0.0       0.0      0.0         0.0   
562             0.0              0.0       0.0      0.0         0.0   
563             0.0              0.0       0.0      0.0         0.0   
564             0.0              0.0       0.0      0.0         0.0   
565             0.0              0.0       0.0      0.0         0.0   

     alternative metal  alternative rock  american folk revival  art pop  \
0                  0.0               0.0         

'\n# Create a pair plot\nsns.set(style="whitegrid")\ng = sns.pairplot(data=data1, hue="top genre", corner=True, palette=\'husl\')\nplt.show()\n'